# Corpus and Dictionary Preparation

In [1]:
import os
import string
import tarfile
import random
import topic_model
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import bigrams
from nltk import trigrams
from gensim import corpora
from gensim.models import Phrases
from gensim.models import phrases

In [2]:
pd.set_option('display.max_info_columns',200)
pd.set_option('display.max_colwidth',200)
pd.set_option('display.max_rows', 100)

In [3]:
df = pd.read_csv('data/cleaned_snowball.csv', sep=',', quotechar='"', index_col=0, header=0)
df.head()

,user.description,text,topic,log_followers,log_retweets,engagement_flag,text_token,text_stemmed,text_processed,positive_emo,outrage_emo,net_emo_outrage
0,"Part of the #VastRightWingConspiracy -- thanks, @HillaryClinton. Better candidate: https://www.randpaul.com/ Retweeting not endorsement. Image: TV lover.","#BanTheBox means ""all is #forgiven""/NO MORE federal #NCIC checks when buying guns? Please consider it, Mr. President @POTUS. #forgiving",G,5.252273,0.0,0,"[u'#BanTheBox', u'means', u'""', u'all', u'is', u'#forgiven', u'""', u'/', u'NO', u'MORE', u'federal', u'#NCIC', u'checks', u'when', u'buying', u'guns', u'?', u'Please', u'consider', u'it', u',', u'...","[u'#banthebox', u'mean', u'""', u'all', u'is', u'#forgiven', u'""', u'/', u'no', u'more', u'feder', u'#ncic', u'check', u'when', u'buy', u'gun', u'?', u'pleas', u'consid', u'it', u',', u'mr', u'.', ...","#banthebox mean "" all is #forgiven "" / no more feder #ncic check when buy gun ? pleas consid it , mr . presid @potus . #forgiv",0,0,0
1,War Veteran who loves his country. Never forget those who died in 'Benghazi'. 864th Engineer Battalion (Combat) (Heavy),BOYCOTT NOV 4 @ABC @CMAwards &gt; THREAT TO ALL GUN OWNERS - BOYCOTT HIS MUSIC &gt; @TheTimMcGraw GUN CONTROL EXTREMIST https://t.co/xMuE2o7vV7 …,G,9.936148,0.0,0,"[u'BOYCOTT', u'NOV', u'4', u'@ABC', u'@CMAwards', u'>', u'THREAT', u'TO', u'ALL', u'GUN', u'OWNERS', u'-', u'BOYCOTT', u'HIS', u'MUSIC', u'>', u'@TheTimMcGraw', u'GUN', u'CONTROL', u'EXTREMIST', u...","[u'boycott', u'nov', u'4', u'@abc', u'@cmaward', u'>', u'threat', u'to', u'all', u'gun', u'owner', u'-', u'boycott', u'his', u'music', u'>', u'@thetimmcgraw', u'gun', u'control', u'extremist', u'h...",boycott nov 4 @abc @cmaward > threat to all gun owner - boycott his music > @thetimmcgraw gun control extremist https://t.co/xmue2o7vv7,0,0,0
2,"Retired: cowboy,truck driver Served in the USAF, hobbies include Shooting sports and hunt and camp born in Calif.and sadly left in 1984","Savage: This is the Most Corrupt, Degenerate, Criminal Gov’t in American History https://t.co/qW12kzuBWH Take my guns lead first.",G,4.043051,0.0,0,"[u'Savage', u':', u'This', u'is', u'the', u'Most', u'Corrupt', u',', u'Degenerate', u',', u'Criminal', u'Gov', u'', u't', u'in', u'American', u'History', u'https://t.co/qW12kzuBWH', u'Take', u'my'...","[u'savag', u':', u'this', u'is', u'the', u'most', u'corrupt', u',', u'degener', u',', u'crimin', u'gov', u'', u't', u'in', u'american', u'histori', u'https://t.co/qw12kzubwh', u'take', u'my', u'gu...","savag : this is the most corrupt , degener , crimin gov t in american histori https://t.co/qw12kzubwh take my gun lead first .",0,0,0
3,Bartender...Bouncer...Autobody......Gravedigger.....Supermodal......OG.....Playboy.........:)\r\n\r\n\r\nDAM...NICE...GUY...MAFIA DEMOLITION INSTALATION,Guns N' Roses - November Rain https://t.co/w2pHuG0QBa via @YouTube,G,6.107023,0.0,0,"[u'Guns', u'N', u""'"", u'Roses', u'-', u'November', u'Rain', u'https://t.co/w2pHuG0QBa', u'via', u'@YouTube']","[u'gun', u'n', u""'"", u'rose', u'-', u'novemb', u'rain', u'https://t.co/w2phug0qba', u'via', u'@youtub']",gun n ' rose - novemb rain https://t.co/w2phug0qba via @youtub,0,0,0
4,Ming doesn't have a trajectory problem.,"I'll say it again: #Tarantino was merely correct. And, to his credit, he's sticking to his rhetorical guns. #BadCops https://t.co/7M4uTpXwyc",G,5.081404,0.0,0,"[u""I'll"", u'say', u'it', u'again', u':', u'#Tarantino', u'was', u'merely', u'correct', u'.', u'And', u',', u'to', u'his', u'credit', u',', u""he's"", u'sticking', u'to', u'his', u'rhetorical', u'gun...","[u""i'll"", u'say', u'it', u'again', u':', u'#tarantino', u'was', u'mere', u'correct', u'.', u'and', u',', u'to', u'his', u'credit', u',', u'he', u'stick', u'to', u'his', u'rhetor', u'gun', u'.', u'...","i'll say it again : #tarantino was mere correct . and , to his credit , he stick to his rhetor gun . #badcop https://t.co/7m4utpxwyc",0,0,0


## What is the breakdown of the tweet distribution

In [4]:
docs = df.text_processed.dropna().tolist()
docs_token = df.text_stemmed.dropna().tolist()
print ("%s gun control tweets, %s gay marriage tweets, and %s climate change tweets" %("{:,}".format(df[df.topic == 'G'].count()[0]), "{:,}".format(df[df.topic == 'M'].count()[0]), "{:,}".format(df[df.topic == 'C'].count()[0])))
print ("%s tweets for LSI" %("{:,}".format(len(docs))))

144,485 gun control tweets, 177,177 gay marriage tweets, and 231,423 climate change tweets
553,086 tweets for LSI


### Clean up the text some more

In [5]:
docs[831]

'ridicul :-) https://t.co/btphogh62c'

In [6]:
keep = set(['!','?'])
stop = set(stopwords.words('english'))
remove = set([x for x in list(string.punctuation) if x not in keep])
stop.update(remove)
stop.update(['',' ','  '])
docs = [[d for d in doc.split(" ") if d not in stop] for doc in docs]

# We want to capture any phrases like "New York" or "New York Times" so we make phrases of phrases for
# commonly co-occurring words to be "New_York" and "New_York_Times" respectively.
doc_phrases = phrases.Phraser(Phrases(docs))
docs = list(doc_phrases[docs])
doc_phrases = phrases.Phraser(Phrases(docs))
docs = list(doc_phrases[docs])

# Shuffle the docs ahead of time for training.
random.shuffle(docs)

#Create bigrams and trigrams of the whole corpus
docs_bigrams = [[' '.join([d[0],d[1]]) for d in bigrams(doc)] for doc in docs]
docs_trigrams = [[' '.join([d[0],d[1]]) for d in trigrams(doc)] for doc in docs]

In [7]:
docs[831]

[u'@emilyminno',
 u"i'm",
 u'realli_enjoy',
 u'read',
 u'wed',
 u'googl',
 u'doc',
 u'fave',
 u'note',
 u'section',
 u'seem_like',
 u'b',
 u'love',
 u'space']

## Prepare the dictionaries and topic corpuses

The LSI model did not produce any useful topics unfortunately.

In [9]:
# UNIGRAMS
# turn our tokenized documents into a id <-> term dictionary
unigrams_dictionary = corpora.Dictionary(docs)
# convert tokenized documents into a document-term matrix
unigrams_corpus = [unigrams_dictionary.doc2bow(doc) for doc in docs]

# BIGRAMS
# turn our tokenized documents into a id <-> term dictionary
bigrams_dictionary = corpora.Dictionary(docs_bigrams)
# convert tokenized documents into a document-term matrix
bigrams_corpus = [bigrams_dictionary.doc2bow(doc) for doc in docs_bigrams]

# TRIGRAMS
# turn our tokenized documents into a id <-> term dictionary
trigrams_dictionary = corpora.Dictionary(docs_trigrams)
# convert tokenized documents into a document-term matrix
trigrams_corpus = [trigrams_dictionary.doc2bow(doc) for doc in docs_trigrams]


## Save EVERYTHING!

In [10]:
unigrams_dictionary.save('models/unigrams_dictionary.pkl', pickle_protocol=2)
bigrams_dictionary.save('models/bigrams_dictionary.pkl', pickle_protocol=2)
trigrams_dictionary.save('models/trigrams_dictionary.pkl', pickle_protocol=2)

In [11]:
# For each of the dictionary files in the models folder gzip them and ensure it worked, then delete the original
os.chdir('models/')
for fn in os.listdir('.'):
    if 'dictionary' in str(fn):
        tar = tarfile.open(fn + '.gz', "w:gz")
        tar.add(fn)
        tar.close()
        if os.path.isfile(fn + '.gz'):
            os.remove(fn)
os.chdir('..')

In [13]:
corpora.MmCorpus.serialize('models/unigrams_corpus.mm', unigrams_corpus)
corpora.MmCorpus.serialize('models/bigrams_corpus.mm', bigrams_corpus)
corpora.MmCorpus.serialize('models/trigrams_corpus.mm', trigrams_corpus)